In [25]:
from pycuda.compiler import *
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.scan import InclusiveScanKernel
import pycuda.gpuarray as gpuarray
import numpy

In [26]:
mod = SourceModule('''
__global__ void cs(int* src, int* dst,int* len)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j = blockIdx.y*blockDim.y + threadIdx.y;
    if (i < len[0])
    {
        if (j<i)
        {
            dst[i] = dst[i] + src[j];
        }
    }
}
''')
cumsum = mod.get_function('cs')

In [27]:
src = numpy.arange(32).astype(numpy.int32)
dst = numpy.copy(src)
len_arr = numpy.array([32]).astype(numpy.int32)

In [28]:
#scan = InclusiveScanKernel(numpy.int32,'a+b')

In [29]:
src_gpu = gpuarray.to_gpu(src)
dst_g = gpuarray.to_gpu(dst)
#scan(src_gpu, dst_g) 

In [30]:
cumsum(cuda.In(src), cuda.InOut(dst),cuda.In(len_arr), block=(32,32,1), grid=(1,1))

In [31]:
dst

array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       33, 34, 35, 36, 41, 43, 44, 45, 49, 50, 53, 54, 55, 56, 57])

In [32]:
numpy.cumsum(src)

array([  0,   1,   3,   6,  10,  15,  21,  28,  36,  45,  55,  66,  78,
        91, 105, 120, 136, 153, 171, 190, 210, 231, 253, 276, 300, 325,
       351, 378, 406, 435, 465, 496], dtype=int32)